In [1]:
%matplotlib inline


In [6]:
import netCDF4
import numpy as np
import pandas as pd
import rasterio

In [176]:
# Birng the AEM data into memory
cond_path = r"C:\Users\PCUser\Desktop\AEM\rjmcmc\EastKimberley_rjmcmc.nc"

rj_dat = netCDF4.Dataset(cond_path, 'r')

In [177]:
coords = np.column_stack((rj_dat['easting'][:].data,
                         rj_dat['northing'][:].data))

In [196]:
# Ugly raster sampling

infile = r"C:\Users\PCUser\Desktop\EK_data\Interp\KeepWaterTable\WaterTable_31Oct\KR_wt_mAHD_regKrig_1000ml.tif"

wt_src = rasterio.open(infile)


wt_poly = box(wt_src.bounds.left, wt_src.bounds.bottom,
              wt_src.bounds.right, wt_src.bounds.top)


inds = []

for i in range(coords.shape[0]):
    if Point(coords[i]).within(poly):
        inds.append(i)

inds = np.array(inds)      

wt_coords = coords[inds]
        
# Now sample the water table elevation and calculate depth

wt_elevs = np.zeros(shape = wt_coords.shape[0],
                    dtype = np.float64)


for i in range(len(wt_coords)):
    wte = next(wt_src.sample(wt_coords[i:i+1]))[0]
    
    wt_elevs[i] = wte
    

# mask out -9999.

mask = np.isclose(wt_elevs, -9999.)

inds = inds[~mask]

wt_elevs = wt_elevs[~mask]

In [197]:
df_final = pd.DataFrame(columns = ['Depth_from', 'Depth_to', 'easting',
                                  'northing'] + ['cond_' + str(i+1) for i in range(100)])

In [198]:
# Now we have the coordinates we need to extract all the 10m pmaps 

# choose our interval

interval = 10. # We will examine the top 10 metres of the water table

for i, idx in enumerate(inds):
    wt_depth = rj_dat['elevation'][idx] - wt_elevs[i]
    
    # Get the depths
    
    ltd = rj_dat['layer_top_depth'][idx]
    
    top_idx = np.searchsorted(ltd, wt_depth,
                              side = 'right')
    
    bottom_idx  = np.searchsorted(ltd, wt_depth + interval,
                              side = 'right')
    layer_idx = np.arange(top_idx, bottom_idx)
    
    # Get the conductivity bins and sum them along the depth axis
    
    cond_bin_counts = rj_dat['conductivity_bin_count'][idx, layer_inds, :].sum(axis = 0)
    
    # Write this into a dataframe
    
    dat = {'Depth_from': ltd[top_idx], 'Depth_to': ltd[bottom_idx], 'easting': rj_dat['easting'][idx],
           "northing": rj_dat['northing'][idx]}
    
    for i in range(len(cond_bin_counts)):
        dat['cond_' + str(i+1)] = cond_bin_counts[i]
    
    # Append dictionary to data frame
        
    df_final = df_final.append(dat, ignore_index = True)
           
    

In [199]:
df_final.to_csv(r"C:\Users\PCUser\Desktop\EK_data\salinity_mapping\EK_cond_pmap_top10m_saturated_zone.csv",
               index = False)